In [1]:
import os
wd = os.path.normpath(os.getcwd() + '/..')
os.chdir(wd)
os.getcwd()

'/home/hyunsung/Workspace/soccercpd'

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from joblib import Parallel, delayed
from src.myconstants import *

pd.set_option('display.width', 250)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

In this code, we generate the aggregated records (formation period records, role period records, and role assignment records) used in formation clustering, role labeling, and other applications from FGP files.

Unfortunately, since we only share the single match data (ID 17985) due to our company's security issue, the following code cannot regenerate the above records when you only use the shared data, i.e. `17985.ugp`. (That is, you can only check how these records are generated by reading this code.)

Instead, we directly share the `form_periods.pkl`, `role_periods.csv`, and `role_records.csv` (resulting files of applying this code to the entire dataset) so that you can reproduce the results of aforementioned tasks in another code files.

### Merging formation period records

In [3]:
mode = 'noncpd'
form_periods_files = os.listdir(f'data/{mode}/form')
form_periods_files.sort()
form_periods_list = []

for file in form_periods_files:
    path = f'data/{mode}/form/{file}'
    form_periods_list.append(pd.read_pickle(path))

form_periods = pd.concat(form_periods_list, ignore_index=True)
form_periods.to_pickle(f'data/{mode}/form_periods.pkl')
form_periods

,activity_id,form_period,session,start_dt,end_dt,duration,coords,edge_mat
0,12864,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820.0,"[[-706.0, 1597.0], [-1362.0, 686.0], [910.0, 1...","[[0.0, 0.981, 0.953, 0.205, 0.027, 0.814, 0.27..."
1,12868,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820.0,"[[-807.0, -1612.0], [44.0, 199.0], [-373.0, -2...","[[0.0, 0.31, 0.721, 0.997, 0.207, 0.049, 0.115..."
2,12868,2,2,2020-01-01 15:02:00,2020-01-01 15:50:00,2880.0,"[[-928.0, -1915.0], [-112.0, 658.0], [-211.0, ...","[[0.0, 0.169, 0.854, 0.989, 0.126, 0.042, 0.24..."
3,12870,1,1,2020-01-01 13:30:00,2020-01-01 14:19:00,2940.0,"[[-1400.0, 57.0], [439.0, -1071.0], [-283.0, 3...","[[0.0, 0.219, 0.884, 0.097, 0.395, 0.959, 0.34..."
4,12870,2,2,2020-01-01 14:33:00,2020-01-01 15:23:00,3000.0,"[[-1266.0, -146.0], [292.0, -877.0], [-205.0, ...","[[0.0, 0.393, 0.871, 0.067, 0.217, 0.998, 0.28..."
...,...,...,...,...,...,...,...,...
802,9064,2,2,2020-01-01 16:02:00,2020-01-01 16:51:00,2940.0,"[[-1424.0, 565.0], [-1553.0, -609.0], [61.0, -...","[[0.0, 0.967, 0.411, 0.062, 0.026, 0.04, 0.865..."
803,9281,1,1,2020-01-01 15:00:00,2020-01-01 15:47:00,2820.0,"[[-1437.0, 113.0], [-1109.0, -959.0], [1523.0,...","[[0.0, 0.969, 0.053, 0.032, 0.028, 0.201, 0.98..."
804,9281,2,2,2020-01-01 16:03:00,2020-01-01 16:51:00,2880.0,"[[-1253.0, -89.0], [-1091.0, -1035.0], [1465.0...","[[0.0, 0.934, 0.044, 0.032, 0.059, 0.194, 0.97..."
805,9285,1,1,2020-01-01 14:01:00,2020-01-01 14:48:00,2820.0,"[[-638.0, 1741.0], [-1425.0, 599.0], [-1561.0,...","[[0.0, 0.986, 0.243, 0.315, 0.159, 0.738, 0.23..."


### Generating role period records

In [4]:
mode = 'noncpd'
role_period_cols = [
    LABEL_ACTIVITY_ID, LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_ROLE_PERIOD,
    LABEL_SESSION, LABEL_START_TIME, LABEL_END_TIME
]
activity_records = pd.read_csv('data/activity_records.csv', header=0, encoding='utf-8-sig')
form_periods = pd.read_pickle(f'data/{mode}/form_periods.pkl')

In [5]:
def generate_role_period_records(fgp_file, mode='gseg_avg'):
    activity_id = int(fgp_file.split('.')[0])
    fgp_path = f'data/{mode}/fgp/{fgp_file}'
    fgp_df = pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')

    grouped = fgp_df.groupby(LABEL_ROLE_PERIOD)
    match_role_periods = grouped[[LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_SESSION]].first()
    match_role_periods[LABEL_ACTIVITY_ID] = activity_id
    match_role_periods[LABEL_START_TIME] = grouped[LABEL_GAMETIME].min().apply(lambda x: int(x[:2]))
    match_role_periods[LABEL_END_TIME] = grouped[LABEL_GAMETIME].max().apply(lambda x: int(x[:2]))
    match_role_periods.reset_index(inplace=True)

    return match_role_periods[role_period_cols]

In [6]:
fgp_files = os.listdir(f'data/{mode}/fgp')
role_period_list = Parallel(n_jobs=50)(
    delayed(generate_role_period_records)(f, mode) for f in tqdm(fgp_files)
)
role_periods = pd.concat(role_period_list, axis=0, ignore_index=True).astype(int)
role_periods.sort_values([LABEL_ACTIVITY_ID, LABEL_ROLE_PERIOD], inplace=True)
role_periods

100%|██████████| 456/456 [00:04<00:00, 111.15it/s]


,activity_id,player_period,form_period,role_period,session,start_time,end_time
3040,1879,1,1,1,1,0,5
3041,1879,1,1,2,1,5,10
3042,1879,1,1,3,1,10,15
3043,1879,1,1,4,1,15,20
3044,1879,1,1,5,1,20,25
...,...,...,...,...,...,...,...
7077,22875,4,2,18,2,33,38
7078,22875,4,2,19,2,38,43
7079,22875,4,2,20,2,43,48
7080,22875,4,2,21,2,48,53


In [7]:
role_periods.to_csv(f'data/{mode}/role_periods.csv', index=False)

### Generating and merging role assignment records

In [8]:
role_record_cols = [
    LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_SESSION, LABEL_DATETIME, LABEL_BASE_ROLE
]

In [7]:
def generate_role_records(fgp_file, mode='gseg_avg', freq=1):
    activity_id = int(fgp_file.split('.')[0])
    fgp_path = f'data/{mode}/fgp/{fgp_file}'
    fgp_df = pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')
    fgp_df[LABEL_DATETIME] = fgp_df[LABEL_DATETIME].apply(lambda dt: datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'))

    grouped = fgp_df.groupby([LABEL_PLAYER_ID, LABEL_ROLE_PERIOD], as_index=False)
    role_records = grouped[role_record_cols].first()
    role_records[LABEL_ACTIVITY_ID] = activity_id
    role_records[LABEL_START_DT] = grouped[LABEL_DATETIME].first()[LABEL_DATETIME] - timedelta(seconds=freq)
    role_records[LABEL_END_DT] = grouped[LABEL_DATETIME].last()[LABEL_DATETIME]
    role_records[LABEL_DURATION] = grouped[LABEL_GAMETIME].count()[LABEL_GAMETIME] * freq

    role_records = pd.merge(role_records, form_periods[[LABEL_ACTIVITY_ID, LABEL_FORM_PERIOD, LABEL_COORDS]])
    role_records[LABEL_X] = role_records.apply(lambda x: x[LABEL_COORDS][x[LABEL_BASE_ROLE]-1, 0], axis=1)
    role_records[LABEL_Y] = role_records.apply(lambda x: x[LABEL_COORDS][x[LABEL_BASE_ROLE]-1, 1], axis=1)

    role_records = role_records[HEADER_ROLE_RECORDS].sort_values(
        [LABEL_PLAYER_ID, LABEL_ROLE_PERIOD], ignore_index=True)
    target_path = f'data/{mode}/role/{activity_id}.csv'
    role_records.to_csv(target_path, index=False, encoding='utf-8-sig')

    return target_path

In [15]:
mode = 'gseg_avg'
fgp_files = os.listdir(f'data/{mode}/fgp')
target_paths = Parallel(n_jobs=50)(delayed(generate_role_records)(f, mode) for f in tqdm(fgp_files))
target_paths.sort()
pd.read_csv(target_paths[0], header=0)

100%|██████████| 457/457 [00:08<00:00, 51.63it/s] 


,activity_id,player_period,form_period,role_period,session,start_dt,end_dt,duration,player_id,squad_num,player_name,base_role,x,y
0,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,4779,3,P03,1,-719.0,1625.0
1,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1925,4,P04,2,-1342.0,660.0
2,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1248,7,P07,3,948.0,1576.0
3,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1759,8,P08,4,-180.0,-451.0
4,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1252,9,P09,5,1574.0,-572.0
5,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,5713,10,P10,6,48.0,543.0
6,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,5439,15,P15,7,-1313.0,-551.0
7,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1235,23,P23,8,-982.0,-1634.0
8,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,6088,72,P72,9,584.0,-1632.0
9,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1773,98,P98,10,1383.0,436.0


In [9]:
role_records_files = os.listdir(f'data/{mode}/role')
role_records_files.sort()
role_records_list = []

for file in role_records_files:
    path = f'data/{mode}/role/{file}'
    role_records_list.append(pd.read_csv(path, header=0))

role_records = pd.concat(role_records_list, ignore_index=True)
for col in HEADER_ROLE_RECORDS[:5]:
    role_records[col] = role_records[col].astype(int)

role_records.to_csv(f'data/{mode}/role_records.csv', index=False)
role_records

,activity_id,player_period,form_period,role_period,session,start_dt,end_dt,duration,player_id,squad_num,player_name,base_role,x,y
0,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:05:00,300.0,1235,23,P23,8,-983.0,-1609.0
1,12864,1,1,2,1,2020-01-01 14:05:00,2020-01-01 14:10:00,300.0,1235,23,P23,8,-983.0,-1609.0
2,12864,1,1,3,1,2020-01-01 14:10:00,2020-01-01 14:15:00,300.0,1235,23,P23,8,-983.0,-1609.0
3,12864,1,1,4,1,2020-01-01 14:15:00,2020-01-01 14:20:00,300.0,1235,23,P23,8,-983.0,-1609.0
4,12864,1,1,5,1,2020-01-01 14:20:00,2020-01-01 14:25:00,300.0,1235,23,P23,8,-983.0,-1609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84252,9285,5,2,21,2,2020-01-01 15:49:00,2020-01-01 15:52:00,180.0,1743,13,P13,9,-749.0,-1695.0
84253,9285,4,2,18,2,2020-01-01 15:35:00,2020-01-01 15:40:00,300.0,4740,7,P07,8,1004.0,1717.0
84254,9285,4,2,19,2,2020-01-01 15:40:00,2020-01-01 15:45:00,300.0,4740,7,P07,8,1004.0,1717.0
84255,9285,4,2,20,2,2020-01-01 15:45:00,2020-01-01 15:49:00,240.0,4740,7,P07,8,1004.0,1717.0
